In [ ]:
from IPython.display import Image


In [2]:
'''
--Imports--
BeautifulSoup is used to parse through Html data.
Requests gets us the data from the webpage.
csv is used to output the data we get into a csv.

'''
from bs4 import BeautifulSoup as bs
import requests
import csv
import json


# Scoutfy Data Mining Bootcamp Tutorial

This file will explain everything you need to know for Scoutfy's bootcamp. For this example we will gather data from the US Fencing website. This is a similar proccess to what you will do with the federations you will be assigned. 

The data will be gathered, output as a csv and added to your folder on Google Drive. I prefer VSCode or Jupyter as IDEs feel free to use whichever one you want.

## Questions

Any questions you might have about this or any problems you encounter while writing your own code you can contact Cezar Pekelman (me), Rafael Petry, or Garren Souza on Discord.


# First Step

https://member.usafencing.org/clubs

This is the url for all fencing clubs associated with US Fencing federation. Feel free to click on the website and take a look at it

### Inspect Element

In the website you can right click and inspect element on specific element to look at their code. The key here is to find the class and html tags.

They will usually be in span classes or div classes. For example:

In [3]:
'''
<span> Foo Bar </span>

<div class_= 'Foo'> Hello World </div>

In the case we have the option to filter by class and only get the tags that have that specific class. This is used in most websites.
'''


"\n<span> Foo Bar </span>\n\n<div class_= 'Foo'> Hello World </div>\n\nIn the case we have the option to filter by class and only get the tags that have that specific class. This is used in most websites.\n"

In [26]:
Image('Inspect1.png')

## Output

The output should be a csv that looks something like this:
This csv file is available at this folder.


In [5]:
Image('csv.png')

## Classses and Tags

As you can see highlited on the left we have two 'p' tags and each with a different class. The class with the name of the institution is "lead mb-0" and the class with the address of the club is "adress tight". In the next cells I will show you how to request the data and filtering through it.

In [165]:
# This is standard in every website you will do this. Request and use BeautifulSoup to "Soupify". 
# The output is the code for the whole page
i=1
url = 'http://carolinasrugby.com/clubs/'
req = requests.get(url)
soup = bs(req.content) 
#number_of_pages = soup.find_all(class_='pager-item')
#print(len(number_of_pages))
#name1 = soup.find(class_='clubmap1').find('a')['href'].split('/')[-1]  #this will get you all the information in the tag
name1 = soup.find_all('div',{'class':'clubmap1'})  #this will get you all the information in the tag
ids = []
for name in name1:
    name_ = name.find('a')['href'].split('/')[-1]
    ids.append(name_)
print(ids)

['61948', '62242', '65521', '62239', '61929', '60351', '66037', '61959', '62243', '61945', '61946', '60349', '66633', '61957', '61932', '67933', '62258', '62244', '61955', '61950', '62241', '61926', '61952', '60350', '62259', '80114', '66035', '66632', '78898', '66036', '78899', '78417', '80772', '61934', '62238', '61922', '61923']


In [213]:
url_clube = 'http://carolinasrugby.com/clubprofile/61948/'
req = requests.get(url_clube)
soup = bs(req.content) 
name = soup.find(class_='righty1').find('h1').text
email = soup.find(class_='righty2').find('h2').text
website = soup.find(class_='righty2').find('h3').text
phone = soup.find(class_='righty2').find('h4').text
logo = soup.find(class_='lefttop').find('img')['src']
address = soup.find(class_='dir').find('a')['href']
print(name)
print(email)
print(website)
print(phone)
print(logo)
print(address)

Asheville Iguanas
ashevillerugby@gmail.com
http://www.ashevillerugby.org
7045601024
https://sportlomo-userupload.s3.amazonaws.com/clubLogos/61948/Asheville Rugby Football Club.jpg
https://www.google.com/maps/dir//35.56921,-82.61593


In [154]:
url_address = address
req = requests.get(url_address)
soup = bs(req.content) 
txt = soup.prettify()
find1 = "window.APP_INITIALIZATION_STATE="
find2 = ";window.APP"
i1 = txt.find(find1)
i2 = txt.find(find2, i1+1 )
js = txt[i1+len(find1):i2]
data = json.loads(js)
#print(data)
d = str(data).find('Estados Unidos')
#print(str(data)[d:-1].split('[[')[1].split(']]')[0].split(']')[0].split(','))
address = str(data)[d:-1].split('[[')[1].split(']]')[0].split(']')[0].split(',')
#address = address[1]+address[2]+address[3]+address[4]
print(address)
print(address[3])

['"Closet Bounce"', '"5 Walnut Tree Ridge', ' Asheville', ' NC 28806', ' Estados Unidos"']
 NC 28806


## Looping and Organizing

The data has to be stored in a Python dictionary before it is output as a csv file. I like to organize it in functions because I think that's the easiest and most readable. There are 60 pages at the website so  we will have to loop through all of them. Our goal is to find as much as we can from this list:

**It is very important that everything matches what it is on Podio**

#### Data you will find in the website

* Name of institution
* Acronym of institution
* Logo
* Address
* City - State
* Federation it's afiliated to
* Email
* Phone Number
* Website
* Sports
* Sponsors
* Social Media

Not always we will be able to get all this information, but it's important to get everything available on the website.

#### Other Data you will need to add manually

* Country
* Hiererchical Affiliation (USA Fencing here)
* Recorded by (your name as it appears on Podio)
* Recorded date




In [201]:
def find_name(soup):
   name = soup.find(class_='righty1').find('h1')
   if name is None:
       return None
   return name.text

In [241]:
def find_lat(soup):
       address = soup.find(class_='dir').find('a')['href']
       if address is None:
        return None
       return address

In [223]:
def find_address(address):
      if address is None:
           return None
      return address

In [204]:
def find_city(city):
    '''
    For city the format we use in the database is always City - Two letter acronym for state. It is important that cities
    always follow this format. Example:
    Los Angeles - CA
    New York - NY
    Baltimore - MD
    '''
    if city is None:
        return None
    return city[2]
            

In [205]:
def find_state(state):
   if state is None:
        return None
   return state[2]

In [206]:
'Not used'
def state_abbreviation(state):
    if state is None:
        return None
    states = json.loads(open('states.json').read())
    states_dictionary = {}
    for key, value in states.items():
           states_dictionary[str(value).replace(' ','')] = key
    return states_dictionary.get(str(state).replace(' ',''))

### Notes

Email, phone number and website are all in tight classes in the webpage inside of p tags. Some clubs have all 3 pieces of data, some have 2 and some have none. We will have to do some checks to differentiare if its an email, phone number or website

### Constants

Some values will be the same for every club, like : country, affiliation, recorded by. Add them as a constant string in each of the dictionaries so they will be in the output csv. There's an example in main.

In [207]:
def find_phone(soup):
      phone = soup.find(class_='righty2').find('h4')
      if phone is None:
             return None
      return phone.text


In [208]:
def find_email(soup):
   email = soup.find(class_='righty2').find('h2')
   if email is None:
       return None
   return email.text


In [209]:
def find_website(soup):
      website = soup.find(class_='righty2').find('h3')
      if website is None:
             return None
      return website.text


In [210]:
def find_logo(soup):
      logo = soup.find(class_='lefttop').find('img')
      if logo is None:
             return None
      return logo['src']

In [211]:
def find_acronym(soup):
    text = soup.find('span',{'class':'views-field-display-name'}).text.strip()
    index = text.find('(') # find the first instance of '(' because we only want the name for now
    if index == -1:
        return None
    return text[index + 1:-1]


In [258]:
def find_address_google_map(soup,url_address):
        if url_address is None:
            return None
        req = requests.get(url_address)
        soup = bs(req.content)
        txt = soup.prettify()
        find1 = "window.APP_INITIALIZATION_STATE="
        find2 = ";window.APP"
        i1 = txt.find(find1)
        i2 = txt.find(find2, i1+1 )
        js = txt[i1+len(find1):i2]
        data = json.loads(js)
        d = str(data).find('Estados Unidos')
        if d == -1:
            return None
        address = str(data)[d:-1].split('[[')[1].split(']]')[0].split(']')[0].split(',')
        print(d)
        return address

In [ ]:
url = 'http://carolinasrugby.com/clubs/'
req = requests.get(url)
soup = bs(req.content) 

my_dict = {}
all_clubs = soup.find_all('div',{'class':'clubmap1'})  #this will get you all the information in the tag
ids = []
for clube in all_clubs:
    clube_ = clube.find('a')['href'].split('/')[-1]
    ids.append(clube_)
for id in ids:
     url_clube = 'http://carolinasrugby.com/clubprofile/'+str(id)+'/'
     req = requests.get(url_clube)
     soup1 = bs(req.content) 
     name = find_name(soup1)
     addr = find_lat(soup1)
     address = find_address_google_map(soup1,addr)
     if address is not  None:
           city = address[2]
     logo = find_logo(soup1)
     phone = find_phone(soup1)
     email = find_email(soup1)
     website = find_website(soup1)
     
     print(name)
     print(address)
     print(logo)
     print(phone)
     print(email)
     print(website)
     print(city)


In [285]:
def main():
    #this list will store every dictionary of each club
    my_list = []
    
    #used for iterating over pages. There are 60 pages of data  
    
    for i in range (1,129):   #1 to 129
       
        #requesting the data
        url = 'https://new.uschess.org/club-search-and-affiliate-directory?display_name=&proximity%5Bcity%5D=&proximity%5Bstate_province_id%5D=&proximity%5Bvalue%5D=&proximity%5Bdistance%5D=&proximity%5Bdistance_unit%5D=miles&page=' + str(i)
        req = requests.get(url)
        soup = bs(req.content)
        
        # Every club is in a "tr" tag. So I will separate each club first to make it easier
        all_clubs = soup.find_all(class_='views-row')
        
        for row in all_clubs[1:]:
            # This dict will store the information of each club
            my_dict = {}
          
            my_dict['name'] = find_name(row)
        
            my_dict['acronym']= find_acronym(row)
         #   my_dict['address'] = find_address(row)
            my_dict['city'] = find_city(row)
            my_dict['state'] =find_state(my_dict['city'])

            my_dict['state'] = state_abbreviation(my_dict['state'])
            my_dict['phone'] = find_phone(row)
            my_dict['email'] = find_email(row)
            my_dict['website'] = find_website(row)
            
            #this are all the constants that must be added
            my_dict['country'] = 'United States of America'
            my_dict['Affilated'] = 'USA Chess'
            my_dict['Recorded By'] = 'Elone Sampaio'
            my_dict['Recorded Date'] = '09/01/2022'

            my_list.append(my_dict) 
            
        
        i = i + 1
    
    return my_list
        
res = main()

In [286]:
res # this is what the output looks like as a list of dictionaries

[{'name': 'Across the Board Chess Alliance (A7362767)',
  'acronym': 'A7362767',
  'city': '',
  'state': None,
  'phone': None,
  'email': None,
  'website': None,
  'country': 'United States of America',
  'Affilated': 'USA Chess',
  'Recorded By': 'Elone Sampaio',
  'Recorded Date': '09/01/2022'},
 {'name': 'ACTIVE CHESS CENTER of ASIA (A6013740)',
  'acronym': 'A6013740',
  'city': '',
  'state': None,
  'phone': None,
  'email': None,
  'website': None,
  'country': 'United States of America',
  'Affilated': 'USA Chess',
  'Recorded By': 'Elone Sampaio',
  'Recorded Date': '09/01/2022'},
 {'name': 'ACTIVE LEARNING/USACHESS (A6015562)',
  'acronym': 'A6015562',
  'city': 'SPRING, Texas',
  'state': 'TX',
  'phone': None,
  'email': 'mborchelt@usachess.com',
  'website': None,
  'country': 'United States of America',
  'Affilated': 'USA Chess',
  'Recorded By': 'Elone Sampaio',
  'Recorded Date': '09/01/2022'},
 {'name': 'ACTIVE PIECES (A6036164)',
  'acronym': 'A6036164',
  'city':

## Exporting to csv

If you have a list of dictionaries it is very easy to export to csv. The first row on the csv will be the keys of the dictionary, the other rows will be the values of each dictionary.

Google collab has some problems with writing csv. If you want you can run it in your machine and copy the csv to your google drive folder.

In [287]:
keys = res[0].keys()


with open('US_chess.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = keys
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(res)
    
csvfile.close()